In [78]:
import pandas as pd
import numpy as np
import time
import datetime

dimension_path = "../DimensionsPre/"
data_path = "../Data/"

In [79]:
Date_dimension = pd.read_csv(dimension_path+"Date_dimension/Date_dimension.csv")
Mobility_dimension = pd.read_csv(dimension_path+"Mobility_dimension/Mobility_dimension.csv")
Patient_dimension = pd.read_csv(dimension_path+"Patient_dimension/Patient_dimension.csv")
PHU_Location_dimension = pd.read_csv(dimension_path+"PHU_Location_dimension/PHU_Location_dimension.csv")
Special_Measures_dimension = pd.read_csv(dimension_path+"Special_Measures_dimension/Special_Measures_dimension.csv")
Weather_dimension = pd.read_csv(dimension_path+"Weather_dimension/Weather_dimension.csv")

In [80]:
cases = pd.read_csv(data_path+"IndividualCasesInOntario.csv")
cases.head()

,_id,Row_ID,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,Specimen_Date,Age_Group,Client_Gender,Case_AcquisitionInfo,Outcome1,Outbreak_Related,Reporting_PHU_ID,Reporting_PHU,Reporting_PHU_Address,Reporting_PHU_City,Reporting_PHU_Postal_Code,Reporting_PHU_Website,Reporting_PHU_Latitude,Reporting_PHU_Longitude
0,1,1,2020-06-04T00:00:00,2020-06-06T00:00:00,2020-06-06T00:00:00,2020-06-04T00:00:00,50s,FEMALE,CC,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
1,2,2,2020-06-01T00:00:00,2020-06-03T00:00:00,2020-06-03T00:00:00,2020-06-02T00:00:00,30s,MALE,CC,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
2,3,3,2020-05-30T00:00:00,2020-06-02T00:00:00,2020-06-02T00:00:00,2020-05-30T00:00:00,20s,MALE,CC,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
3,4,4,2020-04-15T00:00:00,2020-04-17T00:00:00,2020-04-17T00:00:00,2020-04-16T00:00:00,80s,FEMALE,OB,Resolved,Yes,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
4,5,5,2020-04-18T00:00:00,2020-04-21T00:00:00,2020-04-21T00:00:00,2020-04-18T00:00:00,20s,MALE,OB,Resolved,Yes,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893


In [81]:
cases = cases[(cases["Reporting_PHU_City"]=="Toronto")|(cases["Reporting_PHU_City"]=="Ottawa")]


In [82]:
cases["Accurate_Episode_Date"] = pd.to_datetime(cases["Accurate_Episode_Date"]).dt.strftime('%Y-%m-%d')
cases["Case_Reported_Date"] = pd.to_datetime(cases["Case_Reported_Date"]).dt.strftime('%Y-%m-%d')
cases["Test_Reported_Date"] = pd.to_datetime(cases["Test_Reported_Date"]).dt.strftime('%Y-%m-%d')
cases["Specimen_Date"] = pd.to_datetime(cases["Specimen_Date"]).dt.strftime('%Y-%m-%d')
cases = cases[(cases["Test_Reported_Date"] < "2021-03-01") & (cases["Test_Reported_Date"] >= "2020-11-01")]
cases = cases[(cases["Reporting_PHU_City"] == "Toronto") | (cases["Reporting_PHU_City"] == "Ottawa")]

cases = cases.fillna({'Outbreak_Related':'Unknown'})[:1000]

In [83]:
cases["Reporting_PHU_City"].isna().sum()

0

In [84]:
fact_table =  pd.DataFrame(columns=['Onset_date_key','Reported_date_key','Test_date_key', 'Specimen_date_key', 'Patient_key', "PHU_location_key","Special_measure_key","Mobility_key","Weather_key","Resolved","Unresolved","Fatal"])

In [85]:
fact_table.head()

,Onset_date_key,Reported_date_key,Test_date_key,Specimen_date_key,Patient_key,PHU_location_key,Special_measure_key,Mobility_key,Weather_key,Resolved,Unresolved,Fatal


In [86]:
#TO REMOVED:df.dropna(axis=0,how='any')
start = time.time()
cases = cases.dropna(axis=0,how='any')
# cases = cases.head()
rows = 0
Resolved,Unresolved,Fatal = 0,0,0
specialMeasureMismatch, mobilityMismatch = 0,0


for idx, row in cases.iterrows():
    rows += 1
    
    #not considering 2021 data
#     if (row["Case_Reported_Date"][:4]!="2020"):
#         continue
    
    if (rows%100 == 0):
        percentage = str(round((rows/len(cases))*100, 2)  )
        remaining = (time.time()-start)*((len(cases)-rows)/rows)
        print("%s/%s - %s%%  --about %s:%s left"%(str(rows),str(len(cases)),percentage,str(int(remaining//60)),str(int(remaining%60))),end = "\r")
    
    if (row["Outcome1"] == "Resolved"):
        Resolved+=1
    elif (row["Outcome1"] == "Not Resolved"):
        Unresolved+=1
    else:
        Fatal+=1

    
#     specialMeasureId = Special_Measures_dimension[(Special_Measures_dimension["Reporting_PHU_id"]==row["Reporting_PHU_ID"])&(Special_Measures_dimension["Start-date"]<=row["Case_Reported_Date"])&(Special_Measures_dimension["End-date"]>=row["Case_Reported_Date"])]["surrogate_key"].values[0]
# d_time = datetime.datetime.strptime(str(datetime.datetime.now().date())+'9:30', '%Y-%m-%d%H:%M')

    try:
        specialMeasureId = Special_Measures_dimension[(Special_Measures_dimension["Reporting_PHU_id"]==row["Reporting_PHU_ID"])&(Special_Measures_dimension["Start-date"]<=row["Case_Reported_Date"])&(Special_Measures_dimension["End-date"]>=row["Case_Reported_Date"])]["surrogate_key"].values[0]
    except IndexError as e:        
        specialMeasureMismatch+=1
        specialMeasureId = None
#         print(error,e)
#         
    try:
        Mobility_dimension[(Mobility_dimension["sub_region_2"]==row["Reporting_PHU_City"])&(Mobility_dimension["date"]==row["Case_Reported_Date"])]["surrogate_key"].values[0]
    except IndexError as e:        
        mobilityMismatch+=1
        Mobility_dimension = None
#         print(error,e)
        
        

    fact_row = [
    Date_dimension[Date_dimension["date"]==row["Accurate_Episode_Date"]]["surrogate_key"].values[0],
    Date_dimension[Date_dimension["date"]==row["Case_Reported_Date"]]["surrogate_key"].values[0],
    Date_dimension[Date_dimension["date"]==row["Test_Reported_Date"]]["surrogate_key"].values[0],
    Date_dimension[Date_dimension["date"]==row["Specimen_Date"]]["surrogate_key"].values[0],
    Patient_dimension[(Patient_dimension["Age_Group"]==row["Age_Group"])&(Patient_dimension["Client_Gender"]==row["Client_Gender"])&(Patient_dimension["Case_AcquisitionInfo"]==row["Case_AcquisitionInfo"])&(Patient_dimension["Outbreak_Related"]==row["Outbreak_Related"])]["surrogate_key"].values[0],
    PHU_Location_dimension[PHU_Location_dimension["Reporting_PHU_ID"]==row["Reporting_PHU_ID"]]["surrogate_key"].values[0],

    specialMeasureId,
    Mobility_dimension,
    Weather_dimension[(Weather_dimension["Station Name"]==row["Reporting_PHU_City"])&(Weather_dimension["Date/Time"]==row["Case_Reported_Date"])]["surrogate_key"].values[0], 
    Resolved,Unresolved,Fatal


    ]
    fact_table.loc[len(fact_table)] = fact_row

   
print("Finished Forming Fact Table Took %s:%s to Finish"%(int((time.time()-start)//60),int((time.time()-start)%60)))
print("Special Measure Mismatch: %s"%specialMeasureMismatch)
print("Mobility Mismatch: %s"%mobilityMismatch)
print("Total Cases: %s"%len(cases))
        
        
        
        
        

Finished Forming Fact Table Took 0:35 to Finish
Special Measure Mismatch: 58
Mobility Mismatch: 0
Total Cases: 2995


In [87]:
# fact_table.to_csv("Covid-19_Tracking_Fact_Table.csv",index=False)
fact_table

,Onset_date_key,Reported_date_key,Test_date_key,Specimen_date_key,Patient_key,PHU_location_key,Special_measure_key,Mobility_key,Weather_key,Resolved,Unresolved,Fatal
0,327,331,331,327,0,0,149,surrogate_key country_region_code country...,697,1,0,0
1,327,331,331,327,1,0,149,surrogate_key country_region_code country...,697,2,0,0
2,331,331,399,329,2,0,149,surrogate_key country_region_code country...,697,3,0,0
3,328,331,329,328,3,0,149,surrogate_key country_region_code country...,697,4,0,0
4,329,330,330,329,4,0,149,surrogate_key country_region_code country...,696,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2990,354,356,356,354,40,0,149,surrogate_key country_region_code country...,722,2621,345,25
2991,354,356,356,354,12,0,149,surrogate_key country_region_code country...,722,2622,345,25
2992,347,356,356,355,12,1,93,surrogate_key country_region_code country...,356,2623,345,25
2993,339,355,355,353,15,0,149,surrogate_key country_region_code country...,721,2624,345,25


In [99]:
fact_table[2990:2991]

,Onset_date_key,Reported_date_key,Test_date_key,Specimen_date_key,Patient_key,PHU_location_key,Special_measure_key,Mobility_key,Weather_key,Resolved,Unresolved,Fatal
2990,354,356,356,354,40,0,149,surrogate_key country_region_code country...,722,2621,345,25


In [96]:
Date_dimension[Date_dimension["surrogate_key"]==356]["date"]

356    2020-12-22
Name: date, dtype: object

In [97]:
Special_Measures_dimension[Special_Measures_dimension["surrogate_key"]==149]

,surrogate_key,Reporting_PHU,Reporting_PHU_id,Title,Description,Keyword_1,Keyword_2,Start-date,End-date
149,149,Toronto Public Health,3895,Lockdown,Grey – Lockdown \n 1) Implement widescale meas...,Public health measures and advice,Lockdown,2020-11-23,2020-12-25


In [101]:
PHU_Location_dimension[PHU_Location_dimension["surrogate_key"]==0]

,surrogate_key,Reporting_PHU_ID,Reporting_PHU,Reporting_PHU_Address,Reporting_PHU_City,Reporting_PHU_Postal_Code,Reporting_PHU_Website,Reporting_PHU_Latitude,Reporting_PHU_Longitude
0,0,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358


In [90]:
fact_table.head()

,Onset_date_key,Reported_date_key,Test_date_key,Specimen_date_key,Patient_key,PHU_location_key,Special_measure_key,Mobility_key,Weather_key,Resolved,Unresolved,Fatal
0,327,331,331,327,0,0,149,surrogate_key country_region_code country...,697,1,0,0
1,327,331,331,327,1,0,149,surrogate_key country_region_code country...,697,2,0,0
2,331,331,399,329,2,0,149,surrogate_key country_region_code country...,697,3,0,0
3,328,331,329,328,3,0,149,surrogate_key country_region_code country...,697,4,0,0
4,329,330,330,329,4,0,149,surrogate_key country_region_code country...,696,5,0,0


In [91]:
fact_table[2:3]

,Onset_date_key,Reported_date_key,Test_date_key,Specimen_date_key,Patient_key,PHU_location_key,Special_measure_key,Mobility_key,Weather_key,Resolved,Unresolved,Fatal
2,331,331,399,329,2,0,149,surrogate_key country_region_code country...,697,3,0,0


In [92]:
fact_table["Special_measure_key"].drop_duplicates()

0      149
23      93
78      94
159    150
166     95
600    147
608    148
Name: Special_measure_key, dtype: object

In [93]:
fact_table[166:167]

,Onset_date_key,Reported_date_key,Test_date_key,Specimen_date_key,Patient_key,PHU_location_key,Special_measure_key,Mobility_key,Weather_key,Resolved,Unresolved,Fatal
166,415,416,416,415,54,1,95,surrogate_key country_region_code country...,782,163,4,0


In [100]:
fact_table["PHU_location_key"].drop_duplicates()

0     0
23    1
Name: PHU_location_key, dtype: object